In [ ]:
! pip install groq --quiet
! pip install langdetect --quiet

In [ ]:
import json
import requests
import traceback
from groq import Groq

In [ ]:
from google.colab import userdata


In [ ]:
from IPython.display import display, Markdown

In [ ]:
def run_python(code: str):
    try:
        local_env = {}
        exec(code, {}, local_env)
        return {"output": str(local_env)}
    except Exception as e:
        return {"error": str(e), "traceback": traceback.format_exc()}

In [ ]:
# def search_osm_location(query: str, limit: int = 3):
#     try:
#         # url = "https://nominatim.openstreetmap.org/search"
#         url = "https://geocode.maps.co/search"
#         params = {
#             "q": query,
#             "format": "json",
#             "limit": limit,
#             "addressdetails": 1 # For detailed address
#         }
#         headers = {      "User-Agent": "MyApp/1.0 (sjbzmn67@gmail.com)"}
#         r = requests.get(url, params=params, headers=headers)
#         r.raise_for_status()
#         return r.json()
#     except Exception as e:
#         return {"error": str(e)}


import urllib.parse
import requests
from langdetect import detect

def search_location_iq(query: str, limit: int = 3):
    try:
        url = "https://us1.locationiq.com/v1/search"
        params = {
            "key": userdata.get('locationIQ'),
            "q": query,
            "format": "json",
            "limit": limit
        }
        headers = {
            "User-Agent": "MyLocationApp/1.0 (sjbzmn67@gmail.com)"
        }

        r = requests.get(url, params=params, headers=headers, timeout=10)
        r.raise_for_status()
        return r.json()

    except Exception as e:
        return {"error": str(e)}


def query_wolfram_alpha(query):
    url = f'http://api.wolframalpha.com/v2/query?input={query}&format=plaintext&output=JSON&appid={userdata.get('wolframalpha')}'

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pods = data.get('queryresult', {}).get('pods', [])
        results = []
        for pod in pods:
            title = pod.get('title', 'No title')
            subpods = pod.get('subpods', [])
            for subpod in subpods:
                text = subpod.get('plaintext', 'No answer found')
                results.append(f"{title}: {text}")
        return results
    else:
        return [f"Error: {response.status_code}"]




def translate(text, target_lang='en'):
    base_url = "https://api.mymemory.translated.net/get"
    source_lang = detect(text)
    params = {
        "q": text,
         "langpair": f"{source_lang}|{target_lang}"
    }

    url = f"{base_url}?{urllib.parse.urlencode(params)}"
    response = requests.get(url)

    data = response.json()

    # Extract translated text
    translated = data.get("responseData", {}).get("translatedText")

    # If API returns alternatives, pick the highest-ranked match
    if not translated and "matches" in data:
        best = max(data["matches"], key=lambda m: m.get("quality", 0))
        translated = best.get("translation")

    return translated


In [ ]:
FUNCTIONS = [
    {
        "name": "run_python",
        "description": "Execute Python code and return the output.",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {"type": "string"}
            },
            "required": ["code"]
        }
    },
    {
      "name": "search_location_iq",
      "description": "Search a location using LocationIQ",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {"type": "string"}
        },
        "required": ["query"]
      }
    },
        {
        "name": "translate",
        "description": "Translate the language to English if you have difficulty understanding it",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {"type": "string"}
            },
            "required": ["text"]
        }
    },
    {
        "name": "query_wolfram_alpha",
        "description": "Query Wolfram Alpha with a complex math question and get its answer.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string"}
            },
            "required": ["query"]
        }
    }
]

In [ ]:
client = Groq(api_key=userdata.get('Groq'))


In [ ]:
System_prompt = """
You are an AI assistant designed to intelligently choose between natural-language responses and function calls.
For function call responses, don't add additional explanations.

You have access to the following functions:
1. run_python(code: string)
   - Use this to perform calculations, data processing, or any operation that requires executing Python code.

2. search_location_iq(query: string)
   - Use this to search for real-world geographic information based on a given query.
   - If the user asks for coordinates or location info, call search_location_iq using ONLY the location name, not a question.
   - Extract the location name from the query.

3. translate(text: string)
   - Use this to translate the language to English if you have difficulty understanding it.

4. query_wolfram_alpha(query: string)
   - Use this to query Wolfram Alpha with a complex math question and get its answer.

GENERAL BEHAVIOR RULES:
- Determine whether the user's request requires using a function.
- If no function is appropriate, respond with normal conversational text.
- Never hallucinate results that should come from a function.
- For tasks involving computation, data manipulation, or running code logic, prefer `run_python`.
- For tasks involving geographic lookup, coordinates, place names, or map data, prefer `search_osm_location` or 'search_location_iq.


"""

In [ ]:
def call_agent(query: str, system_prompt: str):
    messages=[{"role": "user", "content": query}, {"role": "system", "content": System_prompt}]
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages = messages,
        functions=FUNCTIONS,
        function_call="auto"
    )

    message = response.choices[0].message
    print(f"LLM generated output : {message}")
    messages.append(message)
    if message.function_call:
        fn_name = message.function_call.name
        args = json.loads(message.function_call.arguments)

        print(f"🔧\033[96m LLM wants to call function: {fn_name}\033[0m")
        print(f"📥\033[93m Arguments: {args}\033[0m")



        if fn_name == "run_python":
            result = run_python(**args)
        elif fn_name == "search_location_iq":
            result = search_location_iq(**args)
        elif fn_name == "query_wolfram_alpha":
            result = query_wolfram_alpha(**args)
        elif fn_name == "translate":
            result = translate(**args)
        else:
            result = {"error": "Unknown function"}

        print("📤 Tool Output:", result)
        messages.append({
            "role": "function",
            "name": fn_name,
            "content": json.dumps(result)
        })
        second_response = client.chat.completions.create(
            model="moonshotai/kimi-k2-instruct",
            messages=messages
        )

        return second_response.choices[0].message.content

    return message.content


In [ ]:
query = 'Give a python code to print from 1 to 10 and calculate their sum.'
print(call_agent(query,System_prompt))


LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"code":"numbers = list(range(1, 11))\\nfor num in numbers:\\n    print(num)\\nprint(\\"The sum is: \\", sum(numbers))"}', name='run_python'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: run_python
📥 Arguments: {'code': 'numbers = list(range(1, 11))\nfor num in numbers:\n    print(num)\nprint("The sum is: ", sum(numbers))'}
1
2
3
4
5
6
7
8
9
10
The sum is:  55
📤 Tool Output: {'output': "{'numbers': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'num': 10}"}
The code above prints the numbers from 1 to 10 and outputs their sum (55).


In [ ]:
query = 'Give a python code to detect whether an input string is palindrome or not'
print(call_agent(query,System_prompt))

LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"code":"def is_palindrome(s):\\n    return s == s[::-1]\\n\\ndef main():\\n    s = input(\\"Enter a string: \\")\\n    if is_palindrome(s):\\n        print(\\"The string is a palindrome.\\")\\n    else:\\n        print(\\"The string is not a palindrome.\\")\\n\\nif __name__ == \\"__main__\\":\\n    main()"}', name='run_python'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: run_python
📥 Arguments: {'code': 'def is_palindrome(s):\n    return s == s[::-1]\n\ndef main():\n    s = input("Enter a string: ")\n    if is_palindrome(s):\n        print("The string is a palindrome.")\n    else:\n        print("The string is not a palindrome.")\n\nif __name__ == "__main__":\n    main()'}
📤 Tool Output: {'output': "{'is_palindrome': <function is_palindrome at 0x7cd387f86480>, 'main': <function main at 0x7cd387f85300>}"}
Here’s a 

In [ ]:
query = 'A python code to perform addition of an input array from user'
print(call_agent(query,System_prompt))

LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"code":"def add_array():\\n  user_input = input(\\"Enter the array elements separated by space: \\")\\n  array = [int(i) for i in user_input.split()]\\n  result = sum(array)\\n  return result\\nprint(add_array())"}', name='run_python'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: run_python
📥 Arguments: {'code': 'def add_array():\n  user_input = input("Enter the array elements separated by space: ")\n  array = [int(i) for i in user_input.split()]\n  result = sum(array)\n  return result\nprint(add_array())'}
Enter the array elements separated by space: 10 9 6 7 1
33
📤 Tool Output: {'output': "{'add_array': <function add_array at 0x7cd387f87ec0>}"}
Here’s a minimal script that asks the user for space-separated numbers, converts them to a list of integers, and prints their sum:

```python
# Read input from user
number

In [ ]:
query = 'Can you find the coordinate of BUET?'
print(call_agent(query,System_prompt))

LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"query":"BUET"}', name='search_location_iq'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: search_location_iq
📥 Arguments: {'query': 'BUET'}
📤 Tool Output: [{'place_id': '88912671', 'licence': 'https://locationiq.com/attribution', 'osm_type': 'node', 'osm_id': '4113176525', 'boundingbox': ['45.3340739', '45.3740739', '4.6040783', '4.6440783'], 'lat': '45.3540739', 'lon': '4.6240783', 'display_name': 'Buet, Saint-Appolinard, Saint-Étienne, Loire, Auvergne-Rhône-Alpes, Metropolitan France, 42520, France', 'class': 'place', 'type': 'hamlet', 'importance': 0.23339368047608147, 'icon': 'https://locationiq.org/static/images/mapicons/poi_place_village.p.20.png'}, {'place_id': '285277172', 'licence': 'https://locationiq.com/attribution', 'osm_type': 'node', 'osm_id': '1498691547', 'boundingbox': ['46.7978208', '46.8378208',

In [ ]:
query = 'What are some locations near Mohammadpur, Dhaka?'
print(call_agent(query,System_prompt))

LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"query":"Mohammadpur, Dhaka"}', name='search_location_iq'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: search_location_iq
📥 Arguments: {'query': 'Mohammadpur, Dhaka'}
📤 Tool Output: [{'place_id': '245761981', 'licence': 'https://locationiq.com/attribution', 'osm_type': 'way', 'osm_id': '318722840', 'boundingbox': ['23.7516165', '23.7748701', '90.3454034', '90.375249'], 'lat': '23.7636683', 'lon': '90.363244', 'display_name': 'Mohammadpur, Dhaka Metropolitan, Dhaka District, Dhaka Division, 1207, Bangladesh', 'class': 'boundary', 'type': 'administrative', 'importance': 0.39673767833981843, 'icon': 'https://locationiq.org/static/images/mapicons/poi_boundary_administrative.p.20.png'}, {'place_id': '247749343', 'licence': 'https://locationiq.com/attribution', 'osm_type': 'way', 'osm_id': '481245648', 'boundingbox': ['

In [ ]:

query = '日本の首都はどこですか？'
print(call_agent(query,System_prompt))

LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"text":"日本の首都はどこですか？"}', name='translate'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: translate
📥 Arguments: {'text': '日本の首都はどこですか？'}
📤 Tool Output: What is the capital of Japan?
Tokyo


In [ ]:

query = 'Wer gewann den ersten Nobelpreis für Chemie?'
print(call_agent(query,System_prompt))

LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"text":"Wer gewann den ersten Nobelpreis für Chemie?"}', name='translate'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: translate
📥 Arguments: {'text': 'Wer gewann den ersten Nobelpreis für Chemie?'}
📤 Tool Output: Who won the first Nobel Prize in Chemistry?
The first Nobel Prize in Chemistry was awarded to Jacobus Henricus van 't Hoff in 1901.


In [ ]:
query = 'Integrate 1/x'
print(call_agent(query,System_prompt))

LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"query":"integrate 1/x"}', name='query_wolfram_alpha'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: query_wolfram_alpha
📥 Arguments: {'query': 'integrate 1/x'}
📤 Tool Output: ['Indefinite integral: integral1/x dx = log(x) + constant\n(assuming a complex-valued logarithm)', 'Plot: ', 'Plot: ']
The integral of 1/x is:

**ln|x| + C**

where C is the constant of integration.


In [ ]:
query = 'minimize x^4 - 3*x^2 + 2'
print(call_agent(query,System_prompt))

LLM generated output : ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=FunctionCall(arguments='{"query":"minimize x^4 - 3*x^2 + 2"}', name='query_wolfram_alpha'), reasoning=None, tool_calls=None)
🔧 LLM wants to call function: query_wolfram_alpha
📥 Arguments: {'query': 'minimize x^4 - 3*x^2 + 2'}
📤 Tool Output: ['Input interpretation: minimize | x^4 - (3 x^2)×2', 'Global minima: min{x^4 - (3 x^2) 2} = -9 at x = -sqrt(3)', 'Global minima: min{x^4 - (3 x^2) 2} = -9 at x = sqrt(3)', 'Plot: ']
The global minimum of the function \(x^4 - 3x^2 + 2\) is \(-9\), which occurs at \(x = \pm\sqrt{3}\).
